In [1]:
## system
import os

## Math and dataFrame
import numpy as np
import pandas as pd
import scipy
from scipy.sparse import csr_matrix, hstack

## Visualization
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

## string process
import string
import re

## fast text
from gensim.models import Word2Vec
import gensim

In [2]:
## Traditional Machine Learning
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

In [3]:
## Keras
from keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping#, TensorBoard
from keras import backend as K
from keras import optimizers
from keras.optimizers import SGD
from keras import initializers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/opt/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Couldn't import dot_parser, loading of dot files will not be possible.


In [4]:
from multiprocessing import Pool

model = gensim.models.KeyedVectors.load_word2vec_format(
    '../data/pretrained/freebase-vectors-skipgram1000-en.bin', binary=True)

In [5]:
train = pd.read_csv("../data/train_processed.csv")
test = pd.read_csv("../data/test_processed.csv")
display(train[:3])
display(test[:3])

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i m se...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i m really not trying to edit war it s...,0,0,0,0,0,0


,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you ll...
1,0000247867823ef7,from rfc the title is fine as it is imo
2,00013b17ad220c46,sources zawe ashton on lapland


In [6]:
extra_sent = []
path = '../../ZhiHaoSun/'
for f in os.listdir(path):
    print f
    if f[-3:] == 'txt':
        fp = open(path + f, 'r')
        if 'phrase' in f:
            for line in fp:
                extra_sent.append(''.join(line.split('-')[:-1]))
        else:
            for line in f:
                extra_sent.append(' '.join(line.split(' ')[:-1]))
        fp.close()
print len(extra_sent)
print extra_sent[:30]

severe_toxic_phrases.txt
identity_hate_words.txt
toxic_phrases.txt
severe_toxic_words.txt
toxic_words.txt
threat_phrases.txt
insult_words.txt
insult_phrases.txt
identity_hate_phrases.txt
dictionary_builder.ipynb
threat_words.txt
obscene_phrases.txt
obscene_words.txt
35382
['shit die whore die ', 'die die die ', 'small penis faggot ', 'shit fuck u rape ', 'shit faggot ', 'dick shit cock fuck ', 'die faggot ', 'huge penis ', 'shit go suck ', 'shit suck ', 'small penis closedmouth ', 'small penis ', 'nerd faggot suck ', 'suck dick faggot ', 'suck faggot assholes ', 'big nigger dick shit ', 'nigger dick shit ', 'nigger shit ', 'shit jforget fuck shut ', 'suck ur penis ', 'fat shit go fuck ', 'shit face fuck leave ', 'shit jforget fuck ', 'shit fuck ', 'shit moron ', 'suck asyou suck asshole ', 'fuck youa die ', 'die fuckhead fuck ', 'anthony bradbury sucks penis ', 'die fuck ']


In [7]:
ref = "1234567890qwertyuiopasdfghjklzxcvbnm "
def precess_sent(sent):
    return ''.join([c for c in sent if c in ref])

In [8]:
pool = Pool(4)
es = pool.map(precess_sent, extra_sent)
pool.close()
pool.join()
print es[:10]

['shit die whore die ', 'die die die ', 'small penis faggot ', 'shit fuck u rape ', 'shit faggot ', 'dick shit cock fuck ', 'die faggot ', 'huge penis ', 'shit go suck ', 'shit suck ']


In [10]:
extra_sent = es
print es[:3]

['shit die whore die ', 'die die die ', 'small penis faggot ']


In [11]:
def get_sents(s):
    return s.lower().split()

In [12]:
pool=Pool(4)
%time train_sents = pool.map(get_sents, train.comment_text.str.lower())
%time test_sents = pool.map(get_sents, test.comment_text.str.lower())
%time key_sents = pool.map(get_sents, extra_sent)
pool.close()
pool.join()

CPU times: user 3.44 s, sys: 712 ms, total: 4.15 s
Wall time: 10.3 s
CPU times: user 4.18 s, sys: 524 ms, total: 4.7 s
Wall time: 10.1 s
CPU times: user 68 ms, sys: 8 ms, total: 76 ms
Wall time: 174 ms


In [13]:
model = Word2Vec(train_sents+test_sents, size=100, window=5, min_count=5, workers=4)

In [14]:
model.wv['fuck']

array([-0.84303564,  5.505512  ,  2.2178435 ,  0.76723886,  1.9893103 ,
       -0.3016648 , -1.20258   ,  2.4116788 , -1.4143705 , -1.0409048 ,
        1.3660235 , -1.9697036 ,  1.6473902 , -1.4264855 , -1.0103444 ,
        0.551771  ,  0.5629895 ,  1.226251  , -2.647517  ,  2.5511672 ,
       -0.17387967, -1.9184326 , -1.3316419 ,  0.24120224, -0.9159229 ,
        0.74459535, -1.3509506 ,  0.13880473,  0.02293108,  1.1674203 ,
        3.1347609 , -0.28568706, -0.09199201,  0.26504394, -0.1727153 ,
        1.8427153 ,  0.8492475 , -1.4719603 ,  3.152742  ,  2.0603452 ,
       -1.4947088 , -1.7317444 ,  0.42812875, -0.18267067, -1.2561649 ,
       -0.79223883, -0.5449633 , -0.13332109, -1.091051  ,  0.53540707,
       -1.819616  , -1.5251585 , -2.3003364 , -1.5940654 ,  0.515775  ,
        0.3701657 ,  1.7681065 ,  2.529361  ,  0.35616443,  1.2786752 ,
        2.1912544 ,  1.3000209 , -0.13651122,  0.30851024,  0.5720359 ,
       -0.9974373 , -2.7533169 , -1.6598922 , -0.01443377,  2.24

In [17]:
model = gensim.models.KeyedVectors.load_word2vec_format(
    '../word2vec/data/toxic_key_100_5_5.txt')

UnicodeDecodeError: 'utf8' codec can't decode byte 0x80 in position 0: invalid start byte

In [18]:
model = Word2Vec()

In [20]:
model = Word2Vec.load('../word2vec/data/toxic_key_100_5_5.txt')

In [23]:
model.wv['fuck']

array([-1.4991571 ,  3.274917  ,  2.11796   , -0.1371787 ,  0.38146856,
        0.11616096, -0.08705776,  1.3117185 ,  1.3305957 , -0.8239966 ,
       -0.31085697, -1.6831822 ,  2.0815713 , -0.8283071 ,  0.7265748 ,
       -0.6616854 , -0.6549267 , -1.6704346 , -1.4915667 ,  1.0009212 ,
       -1.4669857 , -0.81920934, -0.01954069,  1.0823163 , -1.2473303 ,
        0.6664338 ,  1.5074086 ,  0.943202  ,  0.7938697 ,  1.4987851 ,
        1.7692456 , -1.6320014 , -0.09104973,  2.1379724 , -1.6890212 ,
        2.097513  ,  0.8531938 , -2.1951742 ,  1.6573819 ,  0.6287992 ,
       -1.0059637 , -1.5043186 ,  1.9915301 , -0.7913232 ,  0.2882915 ,
        4.1284676 , -2.7514691 , -0.49848107, -1.4621027 , -1.5649302 ,
        0.04040527, -0.9068644 , -1.0034015 , -1.9706738 , -0.3348441 ,
       -1.149053  ,  2.5514214 ,  3.692237  ,  0.6378545 ,  1.7018992 ,
        2.3214848 ,  2.4040663 , -0.61251014,  0.01965214, -1.751113  ,
        1.1362225 , -1.6848375 , -0.8693423 ,  2.6890762 ,  1.88